In [4]:
from SStack import SStack
import cpca
from cpca import drawer
from DateFormatconvert2 import DateFormatHelper2
from cilin import CilinSimilarity
from queue import Queue,LifoQueue
import collections
import json
import re
import copy
import Levenshtein #字符串的编辑距离
import json
import os
import sys
sys.setrecursionlimit(999)  # set the maximum depth as 5000

cs = CilinSimilarity()
# 获取指定目录下的json文件
def get_files(path='D:\zyt\\azyt\sfx', rule=".json"):
    all = []
    for fpathe,dirs,fs in os.walk(path):   # os.walk获取所有的目录
        for f in fs:
            filename = os.path.join(fpathe,f)
            if filename.endswith(rule):  # 判断是否是".json"结尾
                all.append(filename)
    return all

#用于将地址信息进行对齐统一
def merge_local(fang_360, fang_baidu):
    location = []
    res_list = []
    if fang_360!='' and fang_baidu=='':
        res_list = [0, fang_360]
    elif fang_baidu!='' and fang_360=='':
        res_list = [1, fang_baidu]
    elif fang_360=='' and fang_baidu=='':
        res_list = [4]
    else:
        if fang_360==fang_baidu:
            res_list = [2, fang_360]
        else:
            location = [fang_360,fang_baidu]
            df = cpca.transform(location) #实现地址信息格式的统一
            province = set()
            city = set()
            zone = set()
            address = set()
            for i in range(len(df['省'])):
                if df['省'][i]:
                    province.add(df['省'][i])
                if df['市'][i]:
                    city.add(df['市'][i])
                if df['区'][i]:
                    zone.add(df['区'][i])
                if df['地址'][i]:
                    address.add(df['地址'][i])
            if len(province)>1 or len(city)>1 or len(zone)>1 or len(address)>1: #表示地址描述不一致，需要消歧
                res_list = [3]
            else:
                last_location = ''
                if len(province)==1:
                    last_location = last_location + list(province)[0]
                if len(city)==1:
                    last_location = last_location + list(city)[0]
                if len(zone)==1:
                    last_location = last_location + list(zone)[0]
#                 if len(address)==1:
#                     last_location = last_location + list(address)[0]
                res_list = [2, last_location]
    return res_list

# 用于将日期信息进行对齐统一
def merge_date(fang_360, fang_baidu):
    res_list = []
    date = set()
    if fang_360!='' and fang_baidu=='': #第一个不为空
        formal_date = DateFormatHelper2.str2date(fang_360) #将日期格式进行统一
        res_list = [0, formal_date]
    elif fang_baidu!='' and fang_360=='': #第二个不为空
        formal_date = DateFormatHelper2.str2date(fang_baidu)
        res_list = [1, formal_date]
    elif fang_baidu=='' and fang_360=='': #两个都为空时
        res_list = [4]
    else: #都不为空时比较
        date.add(DateFormatHelper2.str2date(fang_baidu))
        date.add(DateFormatHelper2.str2date(fang_360))
        if len(date)==1: #两个相等时
            res_list = [2, list(date)[0]] #两个相等时
        else:
            year = set()
            month = set()
            day = set()
            for j in list(date):
                if j.find('-')>0:
                    res = j.split('-')
                    m = 1
                    for i in res:
                        if m==1:
                            year.add(i)
                            m += 1
                        elif m==2:
                            month.add(i)
                            m += 1
                        else:
                            day.add(i)
                            m += 1
                else:
                    year.add(j)
            if len(year)>1 or len(month)>1 or len(day)>1:
                res_list = [3] #表示日期不一致，无法对齐，需要消歧
            else:
                last_date = ''
                if len(year)==1:
                    last_date = last_date + list(year)[0]
                if len(month)==1:
                    last_date = last_date + '-' + list(month)[0]
                if len(day)==1:
                    last_date = last_date + '-' + list(day)[0]
                res_list = [2, last_date]
    return res_list #返回对齐合并后的日期

# 两个字符串的比较(看一个字符串是否是另一个字符串的子集)
def string_compile(str1, str2):
    str1 = re.split('_|\,|\/|\，|\、|\-|\ ' ,str1)
    str2 = re.split('_|\,|\/|\，|\、|\-|\ ' ,str2)
    str1_len = len(str1)
    str2_len = len(str2)
    var_set = set(str1)
    return_str = ''
    for i in str2:
        var_set.add(i)
    if len(var_set) == str1_len:
        j = 0 
        for i in str1:
            if j+1 == str1_len:
                return_str = return_str+i
            else:
                return_str = return_str+i+'/'
            j += 1
        return return_str
    elif len(var_set) == str2_len:
        j = 0 
        for i in str2:
            if j+1 == str2_len:
                return_str = return_str+i
            else:
                return_str = return_str+i+'/'
            j += 1
        return return_str
    else:
        return 1
    
# 5个文件都是列表时的对齐
def compile_list(attribute_value_list, baike_name):
    new_attribute_list = []
    for i in attribute_value_list:
        if i!='':
            new_attribute_list.append(i)
    if len(new_attribute_list)>1:
        list_baike1 = new_attribute_list[0]
        list_baike2 = new_attribute_list[1]
        year = set() #年份集合
        list_360_new = []
        list_baidu_new = []
        for dict_info1 in list_baike1: #依次取出360百科文件列表里的元素，对其中是时间的键值进行格式统一(只取年份)
            for key,value in dict_info1.items():
                if key=='时间点' or key=='起始时间':
                    value = get_year(value)
                    if value!='':
                        year.add(value) #将年份添加到集合中,对齐比较时，只看相同的起始时间的内容
                    dict_info1[key] = value
                if key=='结束时间':
                    value = get_year(value)
                    dict_info1[key] = value
            list_360_new.append(dict_info1)
        for dict_info2 in list_baike2: #依次取出百度百科文件列表里的元素，对其中是时间的键值进行格式统一(只取年份)
            for key,value in dict_info2.items():
                if key=='时间点' or key=='起始时间':
                    value = get_year(value)
                    if value!='':
                        year.add(value)
                    dict_info2[key] = value
                if key=='结束时间':
                    value = get_year(value)
                    dict_info2[key] = value
            list_baidu_new.append(dict_info2)
        return_list = [] #最终返回的字典集合
        flag2 = 0 #用于标记是否出现需要消歧的情况(在当前子属性下)
        for i in list(year):
            tmp_list1 = [] #中间过度列表，用于存放两个文件中年份相同的字典
            tmp_list2 = [] #中间过度列表，用于存放两个文件中年份相同的字典
            for dict_info1 in list_360_new:
                for key,value in dict_info1.items():
                    if (key=='起始时间' or key=='时间点') and value == i:
                        tmp_list1.append(dict_info1)
            tmp_list1_copy = copy.deepcopy(tmp_list1) #因为下面在做的过程中会根据对齐消歧的情况不断改变tmp_list1的值，所以先用一个变量保存
            for dict_info2 in list_baidu_new:
                for key,value in dict_info2.items():
                    if (key=='起始时间' or key=='时间点') and value == i:
                        tmp_list2.append(dict_info2)
            if len(tmp_list1)==0 and len(tmp_list2)!=0:
                for q in tmp_list2:
                    return_list.append(q)
            elif len(tmp_list1)!=0 and len(tmp_list2)==0:
                for q in tmp_list1:
                    return_list.append(q)
            else:
                keys = list(tmp_list1[0].keys())
                flag_list = [] #记录360百科文件与百度百科文件的对齐的字典部分
                for content_year_baidu in tmp_list2:
                    flag = 0 #标识是否需要进行对齐操作
                    tmp_dict1 = copy.deepcopy(content_year_baidu) #用另一个中间变量来表示当前遍历到的字典
                    for content_year_360 in tmp_list1:
                        flag5 = 0 #标识结束时间是否不相等
                        flag6 = 0 #标识其他非关键信息是否一样
                        flag7 = 0 #标识是否是时间点
                        flag4 = 0 #标记是否需要消歧
                        temp_dict = {} #用于存储两个文件对齐字典的中间结果
                        createVar = locals() #动态创建全局变量
                        tmp_dict2 = copy.deepcopy(content_year_360) #用另一个中间变量来表示当前遍历到的字典
                        for tmp_key in keys:
                            #这里之所以会只看这些键的值，是因为这些键的值更具有唯一性，先看具有唯一性的键值是否一样，一样才继续进行下面其他键值的比较，不一样则认为两个字典不是需要对齐的块
                            if tmp_key!='时间点' and tmp_key!='起始时间' and tmp_key!='所在单位' and tmp_key!='信息公布权威机关' and tmp_key!='组织机构' and tmp_key!='方向' and tmp_key!='颁奖单位' and tmp_key!='演讲/报告题目' and tmp_key!='事件' and tmp_key!='语录' and tmp_key!='争议起因' and tmp_key!='事件起因' and tmp_key!='教育理念/指导学生' and tmp_key!='专利名称' and tmp_key!='研究成果名称' and tmp_key!='项目名称' and tmp_key!='散文名称' and tmp_key!='论著名称' and tmp_key!='消歧标记' and tmp_key!='来源':
                                createVar['tmp_360_'+tmp_key] = content_year_360[tmp_key] #分别保存两个文件该键值对应的值
                                createVar['tmp_baidu_'+tmp_key] = content_year_baidu[tmp_key]
                            if tmp_key=='所在单位' or tmp_key=='组织机构' or tmp_key=='方向' or tmp_key=='信息公布权威机关' or tmp_key=='颁奖单位' or tmp_key=='演讲/报告题目' or tmp_key=='事件' or tmp_key=='语录' or tmp_key=='争议起因' or tmp_key=='事件起因' or tmp_key=='教育理念/指导学生' or tmp_key=='专利名称' or tmp_key=='研究成果名称' or tmp_key=='项目名称' or tmp_key=='散文名称' or tmp_key=='论著名称':
                                if content_year_360[tmp_key] == content_year_baidu[tmp_key]:
                                    flag = 1
                                elif cs.similarity(content_year_360[tmp_key], content_year_baidu[tmp_key])==1:
                                    flag = 1
                                elif Levenshtein.distance(content_year_360[tmp_key],content_year_baidu[tmp_key])<4:
                                    flag = 1
                                else:
                                    continue
                        if flag==1:
                            specific_key = ['职称', '职位_职称', '所获奖项']
                            for tmp_key in keys:
                                if tmp_key=='时间点' or tmp_key=='起始时间':
                                    temp_dict[tmp_key] = i
                                    if tmp_key=='时间点':
                                        flag7 = 1
                                elif tmp_key=='所在单位' or tmp_key=='信息公布权威机关' or tmp_key=='组织机构' or tmp_key=='方向' or tmp_key=='颁奖单位' or tmp_key=='演讲/报告题目' or tmp_key=='事件' or tmp_key=='语录' or tmp_key=='争议起因' or tmp_key=='事件起因' or tmp_key=='教育理念/指导学生' or tmp_key=='专利名称' or tmp_key=='研究成果名称' or tmp_key=='项目名称' or tmp_key=='散文名称' or tmp_key=='论著名称':
                                    temp_dict[tmp_key] = content_year_360[tmp_key]
                                else:
                                    if tmp_key=='结束时间':
                                        if eval('tmp_360_'+tmp_key)==eval('tmp_baidu_'+tmp_key):
                                            temp_dict[tmp_key] = content_year_360[tmp_key]
                                            for k in specific_key:
                                                if k in keys:
                                                    if content_year_360[k] == content_year_baidu[k]:
                                                        temp_dict[k] = content_year_360[k]
                                                    elif content_year_360[k]!='' and content_year_baidu[k]=='':
                                                        temp_dict[k] = content_year_360[k]
                                                    elif content_year_360[k]=='' and content_year_baidu[k]!='':
                                                        temp_dict[k] = content_year_baidu[k]
                                                    else:
                                                        str_res = string_compile(content_year_360[k], content_year_baidu[k])
                                                        if str_res!=1:
                                                            temp_dict[k] = str_res
                                                        else:
                                                            flag2 += 1
                                                            flag4 = 1
                                                            tmp_dict1['消歧标记'] = flag2
                                                            tmp_dict1['来源'] = source1
                                                            tmp_dict2['消歧标记'] = flag2
                                                            tmp_dict2['来源'] = source2
                                                            flag_list.append(content_year_360)
                                                            break #通过百科或其他方式实现对齐
                                            if flag4 ==1:
                                                break
                                        else:
                                            flag5 = 1
                                    elif tmp_key=='地点':
                                        res = merge_local(eval('tmp_360_'+tmp_key), eval('tmp_baidu_'+tmp_key))
                                        if res==1:
                                            flag2 += 1
                                            flag4 = 1
                                            tmp_dict1['消歧标记'] = flag2
                                            tmp_dict1['来源'] = source1
                                            tmp_dict2['消歧标记'] = flag2
                                            tmp_dict2['来源'] = source2
                                            flag_list.append(content_year_360)
                                            break #未对齐，需要消歧
                                        else:
                                            temp_dict[tmp_key] = res
                                    else:
                                        if tmp_key!='消歧标记' and tmp_key!='来源':
                                            if eval('tmp_360_'+tmp_key) != eval('tmp_baidu_'+tmp_key):
                                                sim = cs.similarity(eval('tmp_360_'+tmp_key), eval('tmp_baidu_'+tmp_key))
                                                str_res = string_compile(eval('tmp_360_'+tmp_key), eval('tmp_baidu_'+tmp_key))
                                                if sim == 1 or str_res!=1:
                                                    if str_res!=1:
                                                        temp_dict[tmp_key] = str_res
                                                    else:
                                                        temp_dict[tmp_key] = eval('tmp_baidu_'+tmp_key) #相似性得分1，即存在于词库中，直接返回
                                                    if '结束时间' in keys:
                                                        if content_year_360['结束时间'] == content_year_baidu['结束时间']:
                                                            temp_dict['结束时间'] = get_year(content_year_360['结束时间'])
                                                        elif content_year_360['结束时间']!='' and content_year_baidu['结束时间']=='':
                                                            temp_dict['结束时间'] = get_year(content_year_360['结束时间'])
                                                        elif content_year_360['结束时间']=='' and content_year_baidu['结束时间']!='':
                                                            temp_dict['结束时间'] = get(content_year_baidu['结束时间'])
                                                        else:
                                                            flag2 += 1
                                                            flag4 = 1
                                                            tmp_dict1['消歧标记'] = flag2
                                                            tmp_dict1['来源'] = source1
                                                            tmp_dict2['消歧标记'] = flag2
                                                            tmp_dict2['来源'] = source2
                                                            flag_list.append(content_year_360)
                                                            break #通过百科或其他方式实现对齐
                                                    else: #表示是时间点,且有除时间点和关键内容外的其他信息相一致，则认为对齐
                                                        flag_list.append(content_year_360)
                                                        temp_dict = content_year_baidu
                                                        if str_res!=1:
                                                            temp_dict[tmp_key] = str_res
                                                        else:
                                                            temp_dict[tmp_key] = eval('tmp_baidu_'+tmp_key)
                                                        break
                                                else:
                                                    if '结束时间' in keys:
                                                        flag6 = 1
                                                    else: #表示时间点情况下需要消歧
                                                        flag2 += 1
                                                        flag4 = 1
                                                        tmp_dict1['消歧标记'] = flag2
                                                        tmp_dict1['来源'] = source1
                                                        tmp_dict2['消歧标记'] = flag2
                                                        tmp_dict2['来源'] = source2
                                                        flag_list.append(content_year_360)
                                                        break
                                            else:
                                                temp_dict[tmp_key] = eval('tmp_baidu_'+tmp_key)
                                                if '结束时间' in keys:
                                                    if content_year_360['结束时间'] == content_year_baidu['结束时间']:
                                                        temp_dict['结束时间'] = content_year_360['结束时间']
                                                    elif content_year_360['结束时间']!='' and content_year_baidu['结束时间']=='':
                                                        temp_dict['结束时间'] = content_year_360['结束时间']
                                                    elif content_year_360['结束时间']=='' and content_year_baidu['结束时间']!='':
                                                        temp_dict['结束时间'] = content_year_baidu['结束时间']
                                                    else:
                                                        flag2 += 1
                                                        flag4 = 1
                                                        tmp_dict1['消歧标记'] = flag2
                                                        tmp_dict1['来源'] = source1
                                                        tmp_dict2['消歧标记'] = flag2
                                                        tmp_dict2['来源'] = source2
                                                        flag_list.append(content_year_360)
                                                        break #通过百科或其他方式实现对齐
                                                else: #表示时间点情况下是对齐的
                                                    flag_list.append(content_year_360)
                                                    temp_dict = content_year_baidu
                                                    break
                                        else:
                                            pass
                            if flag5==1 and flag6==1: #结束时间和非关键其他信息都不相同时
                                pass
                            else:
                                flag_list.append(content_year_360)
                                break
                    if flag4!=0: #先判断是否需要消歧，如果要消歧就直接把需要消歧的两个文件的内容加入返回列表
                        return_list.append(tmp_dict1)
                        return_list.append(tmp_dict2)
                        tmp_list1.remove(content_year_360)
                    else:
                        if flag==0:
                            return_list.append(content_year_baidu)
                        else:
                            if flag5==1 and flag6==1: #第一层循环的一行比较到第二个循环的最后一个时都没有实现对齐
                                return_list.append(content_year_baidu)
                            else:
                                return_list.append(temp_dict)
                                tmp_list1.remove(content_year_360)
                for content_year_360 in tmp_list1_copy: #找出360百科文件中未与百度百科对齐的部分
                    flag = 0
                    for align_content_360 in flag_list:
                        for tmp_key in keys:
                            if tmp_key=='所在单位' or tmp_key=='信息公布权威机关' or tmp_key=='组织机构' or tmp_key=='方向' or tmp_key=='颁奖单位' or tmp_key=='演讲/报告题目' or tmp_key=='事件' or tmp_key=='语录' or tmp_key=='争议起因' or tmp_key=='事件起因' or tmp_key=='教育理念/指导学生' or tmp_key=='专利名称' or tmp_key=='研究成果名称' or tmp_key=='项目名称' or tmp_key=='散文名称' or tmp_key=='论著名称':
                                if content_year_360[tmp_key] == align_content_360[tmp_key]:
                                    flag = 1 #做一个关键信息相等时的标记
                    if flag==0:
                        return_list.append(content_year_360)
        new_attribute_list.append(return_list)
        del new_attribute_list[0]
        del new_attribute_list[0]
    else:
        return attribute_value_list[0]

# 将结果写入到新json文件中
def list_to_newjson(word, res):
    if word=='本科' or word=='硕士研究生' or word=='博士研究生' or word=='博士后':
        align_json['人物履历']['教育经历'][word]=res
    elif word=='任职' or word=='任免辞职':
        align_json['人物履历']['工作经历'][word]=res
    elif word=='学术兼职' or word=='技术工作兼职' or word=='社会兼职':
        align_json['其他职务'][word]=res
    elif word=='学术论著类' or word=='散文类' or word=='承担项目类' or word=='研究成果类' or word=='发明专利类' or word=='人才培养类':
        align_json["主要成果"][word]=res
    else:
        align_json[word]=res

# 只取日期前面的年份
def get_year(date):
    if date!='':
        date = date.strip()
        year = date[:4]
        year = year + '年'
        return year
    return date

# 维护一个键的队列
def preserve_key_queue(key, key_queue):
    if key=='任免_辞职':
        delete = key_queue.get()
        delete = key_queue.get()
    elif key=='社会兼职' or key=='人才培养类' or key=='博士后':
        delete = key_queue.get()
    else:
        pass

# 比较字典型数据里的子属性的值,输入参数是不同百科改子属性值的列表和对应的属性名
def compare(content_list, index, baike_name):
    if index=='起始时间' or index=='结束时间' or index=='出生日期':
        if len(content_list)>1: #当content_list列表里不止一个元素时，继续比较，直到只剩一个元素才返回
            tmp_dict1 = {}
            tmp_dict2 = {}
            tmp_list = []
            if type(content_list[0]).__name__=='str' and type(content_list[1]).__name__=='str':
                res = merge_date(content_list[0], content_list[1])
                if res[0]==2: #标识都不为空且是对齐了的
                    tmp_dict1[index] = res[1]
                    tmp_dict1['来源'] = baike_name[0]+','+baike_name[1]
                    content_list.append(tmp_dict1)
                elif res[0]==4: #表示两个都为空值
                    pass
                elif res[0]==0: #表示content_list[0]不为空，content_list[1]为空
                    tmp_dict1[index] = res[1]
                    tmp_dict1['来源'] = baike_name[0]
                    content_list.append(tmp_dict1)
                elif res[0]==1: #表示content_list[0]为空，content_list[1]不为空
                    tmp_dict1[index] = res[1]
                    tmp_dict1['来源'] = baike_name[1]
                    content_list.append(tmp_dict1)
                else: #标识两个都不为空，且未对齐
                    tmp_dict1[index] = DateFormatHelper2.str2date(content_list[0])
                    tmp_dict1['消歧标志'] = 1
                    tmp_dict1['来源'] = baike_name[0]
                    tmp_dict2[index] = DateFormatHelper2.str2date(content_list[1])
                    tmp_dict2['消歧标志'] = 1
                    tmp_dict2['来源'] = baike_name[1]
                    tmp_list = [tmp_dict1, tmp_dict2]
                    content_list.append(tmp_list)
                del baike_name[0] #注意删除的时候的问题，连续删除两个的话，删除了第1个过后列表长度会减1
                del baike_name[0]
                del content_list[0]
                del content_list[0]
            elif type(content_list[0]).__name__=='str' and type(content_list[1]).__name__=='dict':
                res = merge_date(content_list[0], content_list[1][index])
                if res[0]==2: #标识都不为空且是对齐了的
                    tmp_dict1[index] = res[1]
                    tmp_dict1['来源'] = content_list[1]['来源']+','+baike_name[0]
                    content_list.append(tmp_dict1)
                elif res[0]==4: #表示两个都为空值
                    pass
                elif res[0]==0: #表示content_list[0]不为空，content_list[1]为空
                    pass
                elif res[0]==1: #表示content_list[0]为空，content_list[1]不为空
                    content_list.append(content_list[1])
                else: #标识两个都不为空，且未对齐
                    tmp_dict1[index] = DateFormatHelper2.str2date(content_list[0])
                    tmp_dict1['消歧标志'] = 1
                    tmp_dict1['来源'] = baike_name[0]
                    content_list[1]['消歧标志'] = 1
                    tmp_list = [content_list[1], tmp_dict1]
                    content_list.append(tmp_list)
                del baike_name[0]
                del content_list[0]
                del content_list[0]
            elif type(content_list[0]).__name__=='dict' and type(content_list[1]).__name__=='dict':
                res = merge_date(content_list[0][index], content_list[1][index])
                if res[0]==2: #标识都不为空且是对齐了的
                    tmp_dict1[index] = res[1]
                    tmp_dict1['来源'] = content_list[1]['来源']+','+content_list[0]['来源']
                    content_list.append(tmp_dict1)
                elif res[0]==3:
                    content_list[0]['消歧标志'] = 1
                    content_list[1]['消歧标志'] = 1
                    tmp_list = [content_list[0], content_list[1]]
                    content_list.append(tmp_list)
                else:
                    pass
                del content_list[0]
                del content_list[0]
            elif type(content_list[0]).__name__=='str' and type(content_list[1]).__name__=='list':
                flag = 0
                for i in content_list[1]:
                    res = merge_date(content_list[0], i[index])
                    if res[0]==2: #标识都不为空且是对齐了的
                        i[index] = res[1]
                        i['来源'] = i['来源']+','+baike_name[0]
                        content_list.append(content_list[1])
                        flag = 1
                        break
                    elif res[0]==4: #表示两个都为空值
                        pass
                    elif res[0]==0: #表示content_list[0]不为空，content_list[1]为空
                        pass
                    elif res[0]==1: #表示content_list[0]为空，content_list[1]不为空
                        content_list.append(content_list[1])
                        flag = 1
                        break
                    else: #标识两个都不为空，且未对齐
                        pass
                if flag==1:
                    del baike_name[0]
                    del content_list[0]
                    del content_list[0]
                else:
                    tmp_dict1[index] = DateFormatHelper2.str2date(content_list[0])
                    tmp_dict1['消歧标志'] = 1
                    tmp_dict1['来源'] = baike_name[0]
                    tmp_list = [tmp_dict1]
                    for i in content_list[1]:
                        tmp_list.append(i)
                    content_list.append(tmp_list)
                    del baike_name[0]
                    del content_list[0]
                    del content_list[0]
            elif type(content_list[0]).__name__=='dict' and type(content_list[1]).__name__=='list':
                flag = 0
                for i in content_list[1]:
                    res = merge_date(content_list[0][index], i[index])
                    if res[0]==2:
                        i[index] = res[1]
                        i['来源'] = i['来源']+','+content_list[0]['来源']
                        content_list.append(content_list[1])
                        flag = 1
                        break
                    else:
                        pass
                if flag==1:
                    del content_list[0]
                    del content_list[0]
                else:
                    content_list[0]['消歧标志'] = 1
                    tmp_list = [content_list[0]]
                    for i in content_list[1]:
                        tmp_list.append(i)
                    content_list.append(tmp_list)
                    del content_list[0]
                    del content_list[0]
            elif type(content_list[0]).__name__=='list' and type(content_list[1]).__name__=='dict':
                flag = 0
                for i in content_list[0]:
                    res = merge_date(content_list[1][index], i[index])
                    if res[0]==2:
                        i[index] = res[1]
                        i['来源'] = i['来源']+','+content_list[1]['来源']
                        content_list.append(content_list[0])
                        flag = 1
                        break
                    else:
                        pass
                if flag==1:
                    del content_list[0]
                    del content_list[0]
                else:
                    content_list[1]['消歧标志'] = 1
                    tmp_list = [content_list[1]]
                    for i in content_list[0]:
                        tmp_list.append(i)
                    content_list.append(tmp_list)
                    del content_list[0]
                    del content_list[0]
            elif type(content_list[0]).__name__=='list' and type(content_list[1]).__name__=='list':
                flag_list = []
                for i in content_list[0]:
                    flag = 0
                    for j in content_list[1]:
                        res = merge_date(i[index], j[index])
                        if res[0]==2:
                            i[index] = res[1]
                            i['来源'] = i['来源']+','+j['来源']
                            tmp_list.append(i)
                            flag = 1
                            flag_list.append(j)
                            break
                        else:
                            pass
                    if flag==0:
                        tmp_list.append(i)
                for i in content_list[1]:
                    flag = 0
                    for j in flag_list:
                        if i[index]!=j[index]:
                            pass
                        else:
                            flag = 1
                            break
                    if flag == 0:
                        tmp_list.append(i)
                content_list.append(tmp_list)
                del content_list[0]
                del content_list[0]
            else:
                pass
            compare(content_list, index, baike_name)
        else:
            return content_list[0]
    elif index=='出生地' or index=='原籍':
        if len(content_list)>1: #当content_list列表里不止一个元素时，继续比较，直到只剩一个元素才返回
            tmp_dict1 = {}
            tmp_dict2 = {}
            tmp_list = []
            if type(content_list[0]).__name__=='str' and type(content_list[1]).__name__=='str':
                res = merge_local(content_list[0], content_list[1])
                if res[0]==2: #标识都不为空且是对齐了的
                    tmp_dict1[index] = res[1]
                    tmp_dict1['来源'] = baike_name[0]+','+baike_name[1]
                    content_list.append(tmp_dict1)
                elif res[0]==4: #表示两个都为空值
                    pass
                elif res[0]==0: #表示content_list[0]不为空，content_list[1]为空
                    tmp_dict1[index] = res[1]
                    tmp_dict1['来源'] = baike_name[0]
                    content_list.append(tmp_dict1)
                elif res[0]==1: #表示content_list[0]为空，content_list[1]不为空
                    tmp_dict1[index] = res[1]
                    tmp_dict1['来源'] = baike_name[1]
                    content_list.append(tmp_dict1)
                else: #标识两个都不为空，且未对齐
                    tmp_dict1[index] = content_list[0]
                    tmp_dict1['消歧标志'] = 1
                    tmp_dict1['来源'] = baike_name[0]
                    tmp_dict2[index] = content_list[1]
                    tmp_dict2['消歧标志'] = 1
                    tmp_dict2['来源'] = baike_name[1]
                    tmp_list = [tmp_dict1, tmp_dict2]
                    content_list.append(tmp_list)
                del baike_name[0] #注意删除的时候的问题，连续删除两个的话，删除了第1个过后列表长度会减1
                del baike_name[0]
                del content_list[0]
                del content_list[0]
            elif type(content_list[0]).__name__=='str' and type(content_list[1]).__name__=='dict':
                res = merge_local(content_list[0], content_list[1][index])
                if res[0]==2: #标识都不为空且是对齐了的
                    tmp_dict1[index] = res[1]
                    tmp_dict1['来源'] = content_list[1]['来源']+','+baike_name[0]
                    content_list.append(tmp_dict1)
                elif res[0]==4: #表示两个都为空值
                    pass
                elif res[0]==0: #表示content_list[0]不为空，content_list[1]为空
                    pass
                elif res[0]==1: #表示content_list[0]为空，content_list[1]不为空
                    content_list.append(content_list[1])
                else: #标识两个都不为空，且未对齐
                    tmp_dict1[index] = content_list[0]
                    tmp_dict1['消歧标志'] = 1
                    tmp_dict1['来源'] = baike_name[0]
                    content_list[1]['消歧标志'] = 1
                    tmp_list = [content_list[1], tmp_dict1]
                    content_list.append(tmp_list)
                del baike_name[0]
                del content_list[0]
                del content_list[0]
            elif type(content_list[0]).__name__=='dict' and type(content_list[1]).__name__=='dict':
                res = merge_local(content_list[0][index], content_list[1][index])
                if res[0]==2: #标识都不为空且是对齐了的
                    tmp_dict1[index] = res[1]
                    tmp_dict1['来源'] = content_list[1]['来源']+','+content_list[0]['来源']
                    content_list.append(tmp_dict1)
                elif res[0]==3:
                    content_list[0]['消歧标志'] = 1
                    content_list[1]['消歧标志'] = 1
                    tmp_list = [content_list[0], content_list[1]]
                    content_list.append(tmp_list)
                else:
                    pass
                del content_list[0]
                del content_list[0]
            elif type(content_list[0]).__name__=='str' and type(content_list[1]).__name__=='list':
                flag = 0
                for i in content_list[1]:
                    res = merge_local(content_list[0], i[index])
                    if res[0]==2: #标识都不为空且是对齐了的
                        i[index] = res[1]
                        i['来源'] = i['来源']+','+baike_name[0]
                        content_list.append(content_list[1])
                        flag = 1
                        break
                    elif res[0]==4: #表示两个都为空值
                        pass
                    elif res[0]==0: #表示content_list[0]不为空，content_list[1]为空
                        pass
                    elif res[0]==1: #表示content_list[0]为空，content_list[1]不为空
                        content_list.append(content_list[1])
                        flag = 1
                        break
                    else: #标识两个都不为空，且未对齐
                        pass
                if flag==1:
                    del baike_name[0]
                    del content_list[0]
                    del content_list[0]
                else:
                    tmp_dict1[index] = content_list[0]
                    tmp_dict1['消歧标志'] = 1
                    tmp_dict1['来源'] = baike_name[0]
                    tmp_list = [tmp_dict1]
                    for i in content_list[1]:
                        tmp_list.append(i)
                    content_list.append(tmp_list)
                    del baike_name[0]
                    del content_list[0]
                    del content_list[0]
            elif type(content_list[0]).__name__=='dict' and type(content_list[1]).__name__=='list':
                flag = 0
                for i in content_list[1]:
                    res = merge_local(content_list[0][index], i[index])
                    if res[0]==2:
                        i[index] = res[1]
                        i['来源'] = i['来源']+','+content_list[0]['来源']
                        content_list.append(content_list[1])
                        flag = 1
                        break
                    else:
                        pass
                if flag==1:
                    del content_list[0]
                    del content_list[0]
                else:
                    content_list[0]['消歧标志'] = 1
                    tmp_list = [content_list[0]]
                    for i in content_list[1]:
                        tmp_list.append(i)
                    content_list.append(tmp_list)
                    del content_list[0]
                    del content_list[0]
            elif type(content_list[0]).__name__=='list' and type(content_list[1]).__name__=='dict':
                flag = 0
                for i in content_list[0]:
                    res = merge_local(content_list[1][index], i[index])
                    if res[0]==2:
                        i[index] = res[1]
                        i['来源'] = i['来源']+','+content_list[1]['来源']
                        content_list.append(content_list[0])
                        flag = 1
                        break
                    else:
                        pass
                if flag==1:
                    del content_list[0]
                    del content_list[0]
                else:
                    content_list[1]['消歧标志'] = 1
                    tmp_list = [content_list[1]]
                    for i in content_list[0]:
                        tmp_list.append(i)
                    content_list.append(tmp_list)
                    del content_list[0]
                    del content_list[0]
            elif type(content_list[0]).__name__=='list' and type(content_list[1]).__name__=='list':
                flag_list = []
                for i in content_list[0]:
                    flag = 0
                    for j in content_list[1]:
                        res = merge_local(i[index], j[index])
                        if res[0]==2:
                            i[index] = res[1]
                            i['来源'] = i['来源']+','+j['来源']
                            tmp_list.append(i)
                            flag = 1
                            flag_list.append(j)
                            break
                        else:
                            pass
                    if flag==0:
                        tmp_list.append(i)
                for i in content_list[1]:
                    flag = 0
                    for j in flag_list:
                        if i[index]!=j[index]:
                            pass
                        else:
                            flag = 1
                            break
                    if flag == 0:
                        tmp_list.append(i)
                content_list.append(tmp_list)
                del content_list[0]
                del content_list[0]
            else:
                pass
            compare(content_list, index, baike_name)
        else:
            return content_list[0]
    else: #除开时间、地址的其他情况对齐
        if len(content_list)>1: #当content_list列表里不止一个元素时，继续比较，直到只剩一个元素才返回
            tmp_dict1 = {}
            tmp_dict2 = {}
            tmp_list = []
            if type(content_list[0]).__name__=='str' and type(content_list[1]).__name__=='str':
                res = compare_other(content_list[0], content_list[1])
                if res[0]==2: #标识都不为空且是对齐了的
                    tmp_dict1[index] = res[1]
                    tmp_dict1['来源'] = baike_name[0]+','+baike_name[1]
                    content_list.append(tmp_dict1)
                elif res[0]==4: #表示两个都为空值
                    pass
                elif res[0]==0: #表示content_list[0]不为空，content_list[1]为空
                    tmp_dict1[index] = res[1]
                    tmp_dict1['来源'] = baike_name[0]
                    content_list.append(tmp_dict1)
                elif res[0]==1: #表示content_list[0]为空，content_list[1]不为空
                    tmp_dict1[index] = res[1]
                    tmp_dict1['来源'] = baike_name[1]
                    content_list.append(tmp_dict1)
                else: #标识两个都不为空，且未对齐
                    tmp_dict1[index] = content_list[0]
                    tmp_dict1['消歧标志'] = 1
                    tmp_dict1['来源'] = baike_name[0]
                    tmp_dict2[index] = content_list[1]
                    tmp_dict2['消歧标志'] = 1
                    tmp_dict2['来源'] = baike_name[1]
                    tmp_list = [tmp_dict1, tmp_dict2]
                    content_list.append(tmp_list)
                del baike_name[0] #注意删除的时候的问题，连续删除两个的话，删除了第1个过后列表长度会减1
                del baike_name[0]
                del content_list[0]
                del content_list[0]
            elif type(content_list[0]).__name__=='str' and type(content_list[1]).__name__=='dict':
                res = compare_other(content_list[0], content_list[1][index])
                if res[0]==2: #标识都不为空且是对齐了的
                    tmp_dict1[index] = res[1]
                    tmp_dict1['来源'] = content_list[1]['来源']+','+baike_name[0]
                    content_list.append(tmp_dict1)
                elif res[0]==4: #表示两个都为空值
                    pass
                elif res[0]==0: #表示content_list[0]不为空，content_list[1]为空
                    pass
                elif res[0]==1: #表示content_list[0]为空，content_list[1]不为空
                    content_list.append(content_list[1])
                else: #标识两个都不为空，且未对齐
                    tmp_dict1[index] = content_list[0]
                    tmp_dict1['消歧标志'] = 1
                    tmp_dict1['来源'] = baike_name[0]
                    content_list[1]['消歧标志'] = 1
                    tmp_list = [content_list[1], tmp_dict1]
                    content_list.append(tmp_list)
                del baike_name[0]
                del content_list[0]
                del content_list[0]
            elif type(content_list[0]).__name__=='dict' and type(content_list[1]).__name__=='dict':
                res = compare_other(content_list[0][index], content_list[1][index])
                if res[0]==2: #标识都不为空且是对齐了的
                    tmp_dict1[index] = res[1]
                    tmp_dict1['来源'] = content_list[1]['来源']+','+content_list[0]['来源']
                    content_list.append(tmp_dict1)
                elif res[0]==3:
                    content_list[0]['消歧标志'] = 1
                    content_list[1]['消歧标志'] = 1
                    tmp_list = [content_list[0], content_list[1]]
                    content_list.append(tmp_list)
                else:
                    pass
                del content_list[0]
                del content_list[0]
            elif type(content_list[0]).__name__=='str' and type(content_list[1]).__name__=='list':
                flag = 0
                for i in content_list[1]:
                    res = compare_other(content_list[0], i[index])
                    if res[0]==2: #标识都不为空且是对齐了的
                        i[index] = res[1]
                        i['来源'] = i['来源']+','+baike_name[0]
                        content_list.append(content_list[1])
                        flag = 1
                        break
                    elif res[0]==4: #表示两个都为空值
                        pass
                    elif res[0]==0: #表示content_list[0]不为空，content_list[1]为空
                        pass
                    elif res[0]==1: #表示content_list[0]为空，content_list[1]不为空
                        content_list.append(content_list[1])
                        flag = 1
                        break
                    else: #标识两个都不为空，且未对齐
                        pass
                if flag==1:
                    del baike_name[0]
                    del content_list[0]
                    del content_list[0]
                else:
                    tmp_dict1[index] = content_list[0]
                    tmp_dict1['消歧标志'] = 1
                    tmp_dict1['来源'] = baike_name[0]
                    tmp_list = [tmp_dict1]
                    for i in content_list[1]:
                        tmp_list.append(i)
                    content_list.append(tmp_list)
                    del baike_name[0]
                    del content_list[0]
                    del content_list[0]
            elif type(content_list[0]).__name__=='dict' and type(content_list[1]).__name__=='list':
                flag = 0
                for i in content_list[1]:
                    res = compare_other(content_list[0][index], i[index])
                    if res[0]==2:
                        i[index] = res[1]
                        i['来源'] = i['来源']+','+content_list[0]['来源']
                        content_list.append(content_list[1])
                        flag = 1
                        break
                    else:
                        pass
                if flag==1:
                    del content_list[0]
                    del content_list[0]
                else:
                    content_list[0]['消歧标志'] = 1
                    tmp_list = [content_list[0]]
                    for i in content_list[1]:
                        tmp_list.append(i)
                    content_list.append(tmp_list)
                    del content_list[0]
                    del content_list[0]
            elif type(content_list[0]).__name__=='list' and type(content_list[1]).__name__=='dict':
                flag = 0
                for i in content_list[0]:
                    res = compare_other(content_list[1][index], i[index])
                    if res[0]==2:
                        i[index] = res[1]
                        i['来源'] = i['来源']+','+content_list[1]['来源']
                        content_list.append(content_list[0])
                        flag = 1
                        break
                    else:
                        pass
                if flag==1:
                    del content_list[0]
                    del content_list[0]
                else:
                    content_list[1]['消歧标志'] = 1
                    tmp_list = [content_list[1]]
                    for i in content_list[0]:
                        tmp_list.append(i)
                    content_list.append(tmp_list)
                    del content_list[0]
                    del content_list[0]
            elif type(content_list[0]).__name__=='list' and type(content_list[1]).__name__=='list':
                flag_list = []
                for i in content_list[0]:
                    flag = 0
                    for j in content_list[1]:
                        res = compare_other(i[index], j[index])
                        if res[0]==2:
                            i[index] = res[1]
                            i['来源'] = i['来源']+','+j['来源']
                            tmp_list.append(i)
                            flag = 1
                            flag_list.append(j)
                            break
                        else:
                            pass
                    if flag==0:
                        tmp_list.append(i)
                for i in content_list[1]:
                    flag = 0
                    for j in flag_list:
                        if i[index]!=j[index]:
                            pass
                        else:
                            flag = 1
                            break
                    if flag == 0:
                        tmp_list.append(i)
                content_list.append(tmp_list)
                del content_list[0]
                del content_list[0]
            else:
                pass
            compare(content_list, index, baike_name)
        else:
            return content_list[0]
    
# 根据库比较除时间、地点外其他值是否对齐，接收的参数为两个文件属性的值，以列表形式返回，一个是内容一个是标识
def compare_other(content1, content2):
    res_list = [] #以列表形式返回，一个是内容一个是标识
    if content1!='' and content2=='':
        res_list = [0, content1] #标识第一个不为空，第二个为空
    elif content1=='' and content2!='': 
        res_list = [1, content2] #标识第一个为空，第二个不为空
    elif content1!='' and content2!='':
        if content1==content2:
            res_list = [2, content1]
        else:
            res = cs.similarity(content1, content2)
            if res==1:
                res_list = [2, content1] #标识两个都不为空且是对齐的
            else:
                res_list = [3] #标识两个都不为空，但未对齐
    else:
        res_list = [4] #标识两个都为空
    return res_list
 
'''
# 对非列表且是字典形式的内容进行对齐(这里主要关注基本信息、本科、硕士、博士、博士后这几个模块)
# 接收参数为属性值列表和百科文件名
# 按照时间、地点和其他信息处理
'''
def align_dict(attribute_value_list, baike_name):
    keys = list(attribute_value_list[0].keys()) #得到该属性下的所有键的名字，并放到一个list里
    new_dict = {} #用于存放对齐与未对齐(需消歧)后的内容
    for index in keys:
        value = []
        for i in attribute_value_list:
            value.append(i[index])
        var_baike_name = copy.deepcopy(baike_name)
        res = compare(value, index, var_baike_name)
        new_dict[index] = res
    return new_dict
                                                        
# 对非列表非字典形式的内容对齐(也包括都是列表，但列表里面都是字符串，不是字典的情况)
def string_align(word, content_list):
    new_list = []
    list_to_set = set() #把list里的内容放到集合中，实现自动去重
    for i in content_list:
        if i=='':
            pass
        else:
            for j in i:
                list_to_set.add(j)
    new_list = list(list_to_set)
    return new_list

'''本函数用户判断某个属性对应的值的类型，并根据类型调用相应的函数处理，
   接受参数为维护的栈和队列，以及各百科文件的来源名
   当属性队列为空的时候，返回0
'''
def other_align(st, key_queue, baike_name):
    if not st.is_empty():
        content = set()
        word = st.pop() #弹出当前栈里面顶上的内容
        key_queue.put(word) #键值队列，用于保存前面已出现的键值(目的是方便后面迭代时能准确找到进行到什么地方了)
        index = 0 #index变量，用于在json_file(百科的json文件列表)里找到对应的百科文件
        attribute_value_list = [] #局部列表变量，用于存储不同百科文件当前属性下的值
        for i in baike_name:
            p = json_file[index] #将当前遍历获得的百科文件存入到变量P里面
            key_str = 'p'
            for j in key_queue.queue: #通过维护的键的队列取到对应的键值
                key_str = key_str+'[\''+j+'\']'
            attribute_value_list.append(eval(key_str))
            index += 1
        flag = 0 #标记这几个百科json文件的当前属性值是否为空
        '''
        这里之所以遍历查看的原因是可能五个文件中有些该属性的值为空，但其他文件却为list或dict，
        如果先遍历到为空的文件，也先不管，还是继续遍历，直到找到不为空的，执行相应的操作,
        但一旦遍历到了不为空的，就可以break退出循环，以免重复进行操作
        '''
        for i in attribute_value_list:
            if type(i).__name__ == 'list': #处理百科json文件中该属性下的值如果是列表时的情况，则取出列表里的每一个字典
                flag = 1
                if word=='人物影响' or word=='研究领域': #这两个属性的值没有子属性了，其值直接就是字符串形式，所以单独处理
                    res = string_align(word, attribute_value_list)
                    list_to_newjson(word, res)
                    key = key_queue.get()
                    preserve_key_queue(key, key_queue)
                    other_align(st, key_queue, baike_name)
                else: #其他情况的属性值里还有子属性，此时如果是列表的话也是单独处理
                    res = compile_list(attribute_value_list, baike_name) #返回的是一个列表，列表里面是多个字典
                    list_to_newjson(word,res)
                    key = key_queue.get()
                    preserve_key_queue(key, key_queue) #得到动态维护的键的队列
                    other_align(st, key_queue, baike_name)
                break
            elif type(i).__name__ == 'dict': #处理百科json文件中该属性下的值如果是字典时的情况，这里主要是针对单属性值
                flag = 1
                if word=='基本信息' or word=='本科' or word=='硕士研究生' or word=='博士研究生' or word=='博士后':
                    res = align_dict(attribute_value_list, baike_name)
                    list_to_newjson(word,res)
                    key = key_queue.get()
                    preserve_key_queue(key, key_queue)
                    other_align(st, key_queue, baike_name)
                else: #当不是单属性值时，说明其还不是最外层的属性(也即为父属性)，则将其压入栈中后续出栈时再处理
                    dict_key1 = list(attribute_value_list[0].keys()) #将字典里的键都放入栈中，稍后依次取出进行对齐
                    dict_key1.reverse()
                    for i in dict_key1:
                        st.push(i)
                    other_align(st, key_queue, baike_name)
                break
            else:
                pass
        if flag==0: #表示这几个文件的当前属性都为空
            res = ''
            list_to_newjson(word, res)
            key = key_queue.get()
            preserve_key_queue(key, key_queue)
            other_align(st, key_queue, baike_name)
    else:
        return 0

# 对齐处理的入口函数，接收参数为最外层的属性名列表和对应的百科json文件名列表
def process_entrace(key_list, baike_name):
    tree=lambda:collections.defaultdict(tree)
    global align_json #全局变量，用于存储对齐和未对齐(做标记)的结果
    align_json=tree()
    st = SStack() #用栈存放每一轮的属性
    lq = LifoQueue(maxsize=0) #用后进先出队列用于存放文件操作过程中的中间键值(用于新建新的对齐json)
    key_list.reverse() #属性名列表反向，为了使最后按照正向的方式写入
    for i in key_list:
        st.push(i) #将最开始外层的属性名压入栈中
    res = other_align(st, lq, baike_name) #调用对齐的控制函数
    print("  ",json.dumps(align_json,ensure_ascii=False,indent=4))

if __name__ == "__main__":
    b = get_files(r"./")
    global json_file #全局列表变量，用于存放不同百科json文件的内容
    json_file = []
    baike_name = [] #局部变量，用于存放json百科文件的名字，对应数据来源
    for i in b:
        if i.split('/')[1]!='fang_merge.json' and i.split('/')[1]!='merge.json':
            with open(i) as f: #打开json文件并保存到json文件列表里
                json_file.append(json.load(f))
            baike_name.append(i.split('/')[1].split('.')[0]) #取出百科json文件的文件名
    key_list = list(json_file[0].keys()) #局部变量，用于存放百科json文件的最外层父属性
    process_entrace(key_list, baike_name)

   {'中文名': '方滨兴', '来源': 'fang_360,fang_baidu,fang_wiki,fang_hudong,fang_sougou'}
   {'外文名': 'Bin xing Fang', '来源': 'fang_baidu'}
   {'性别': '男', '来源': 'fang_baidu,fang_wiki,fang_hudong,fang_sougou'}
   {'出生日期': '1960-7-17', '来源': 'fang_360,fang_baidu,fang_wiki,fang_hudong,fang_sougou'}
   {'出生地': '黑龙江省哈尔滨市', '来源': 'fang_360,fang_baidu,fang_wiki,fang_hudong,fang_sougou'}
   {'国籍': '中国', '来源': 'fang_360,fang_baidu,fang_wiki,fang_hudong,fang_sougou'}
   {'民族': '汉族', '来源': 'fang_360,fang_baidu,fang_hudong,fang_sougou'}
   [{'职业': '学者，教授', '消歧标志': 1, '来源': 'fang_hudong'}, {'职业': '国家计算机网络应急技术处理协调中心主任', '消歧标志': 1, '来源': 'fang_sougou,fang_360'}, {'职业': '计算机科学家、信息安全专家、中国工程院院士', '消歧标志': 1, '来源': 'fang_wiki'}, {'职业': '科研工作者', '消歧标志': 1, '来源': 'fang_baidu'}]
   [{'毕业院校': '哈尔滨工业大学，清华大学', '消歧标志': 1, '来源': 'fang_hudong'}, {'毕业院校': '哈尔滨工业大学', '消歧标志': 1, '来源': 'fang_sougou,fang_360,fang_baidu'}]
   {'政治面貌': '中国共产党', '来源': 'fang_360,fang_baidu,fang_wiki,fang_hudong,fang_sougou'}
   [{'代表作品': '《论网络空间主权》、《

e:\anaconda3\lib\re.py:212: FutureWarning: split() requires a non-empty pattern match.
  return _compile(pattern, flags).split(string, maxsplit)


NameError: name 'source1' is not defined

In [1]:
from DateFormatconvert2 import DateFormatHelper2
dict1 = [{'起始时间':'1985年','结束时间':'1988年'},{'起始时间':'1989年','结束时间':'1993年'}]
print("  ",DateFormatHelper2.str2date('1985.10'))

   1985-10
